In [2]:
import numpy as np
import obspy
import glob
from scipy import signal
import math
import matplotlib.pyplot as plt
import sys
import h5py
import os
import pandas as pd
import cartopy

In [3]:
def obspy_filter(data,lowcut,highcut,fs,order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    z, p, k = signal.iirfilter(order, [low, high], btype='band',
                        ftype='butter', output='zpk')
    sos = signal.zpk2sos(z, p, k)
    new_data=signal.sosfilt(sos, data)
    return new_data

def locate_average(data,length,mode='maximum'):
    
    ave=np.convolve(data,np.ones(length)/length,mode='valid')
        
    if mode =='maximum':
        return np.max(ave)
    elif mode =='minimum':
        return np.min(ave)
    else: 
        raise 'error'

def cal_snr(data,st,lt,fs):
    signal=locate_average(data,st*fs,'maximum')
    noise=locate_average(data,lt*fs,'minimum')
    snr=10*np.log10(signal/noise)
    return snr

In [4]:
npys=np.sort(glob.glob('/home/yaolinm/Projects/Japan/Miao22/Figure_3/SNR_new/*_hz_*'))
indexes=[]
mags=[]
for npy in npys:
    indexes.append(os.path.basename(npy)[5:9])   
    mags.append(os.path.basename(npy)[11:14])

In [4]:
for n in range(len(indexes)):
    
    print(n)
    title='index'+str(indexes[n]).zfill(4)+'_M'+mags[n]
    file=glob.glob('/nfs/turbo/lsa-zspica/work/yaolinm/for_jiuxun/prem_model_events/index'+str(indexes[n]).zfill(4)+'/*.h5')[0]

    data=h5py.File(file,'r')['DAS']
    snrs=np.zeros(2745)
    for i in range(2745):
        trdata=signal.detrend(data[i*5,:],type='constant')
        snrs[i]=cal_snr(np.abs(trdata),1,5,100)
        np.save('/home/yaolinm/Projects/Japan/Miao22/Figure_3/SNR_new/'+title+'_raw_snr.npy',snrs)

In [6]:
npys

array(['/home/yaolinm/Projects/Japan/Miao22/Figure_3/SNR_new/index0094_M2.3_1_8_hz_snr.npy',
       '/home/yaolinm/Projects/Japan/Miao22/Figure_3/SNR_new/index0110_M2.2_1_8_hz_snr.npy',
       '/home/yaolinm/Projects/Japan/Miao22/Figure_3/SNR_new/index0119_M6.3_1_8_hz_snr.npy',
       '/home/yaolinm/Projects/Japan/Miao22/Figure_3/SNR_new/index0373_M2.5_1_8_hz_snr.npy',
       '/home/yaolinm/Projects/Japan/Miao22/Figure_3/SNR_new/index0378_M3.3_1_8_hz_snr.npy',
       '/home/yaolinm/Projects/Japan/Miao22/Figure_3/SNR_new/index0474_M2.8_1_8_hz_snr.npy',
       '/home/yaolinm/Projects/Japan/Miao22/Figure_3/SNR_new/index0744_M4.0_1_8_hz_snr.npy',
       '/home/yaolinm/Projects/Japan/Miao22/Figure_3/SNR_new/index0746_M3.0_1_8_hz_snr.npy',
       '/home/yaolinm/Projects/Japan/Miao22/Figure_3/SNR_new/index0834_M4.6_1_8_hz_snr.npy',
       '/home/yaolinm/Projects/Japan/Miao22/Figure_3/SNR_new/index0873_M5.0_1_8_hz_snr.npy',
       '/home/yaolinm/Projects/Japan/Miao22/Figure_3/SNR_new/index1253